In [ ]:
# 0.0 import packages

import shutil
import os
import mido
import numpy as np
import matplotlib.pyplot as plot

In [ ]:
# midi value for instruments
MIDI_OFFSET = 0

# mappings for our own training (9)
KICK = 0
SNARE = 1
HH_CLOSED = 2
HH_OPEN = 3
RIDE = 4
TOM_1 = 5
TOM_2 = 6
TOM_3 = 7
CRASH = 8
NUM_FEATS = 9

In [ ]:
# 1.0 helper functions
# code modified from:
# https://medium.com/analytics-vidhya/convert-midi-file-to-numpy-array-in-python-7d00531890c

# grow an array down keeping the distance between values proportional
def grow_array_proportionally(midi_array, target_resize):
    resized_array = np.zeros((target_resize, midi_array.shape[1]))
    ratio = midi_array.shape[0] / target_resize
    # iterate through each time-step
    for t in range(target_resize): 
        # for each value
        for i in range(midi_array.shape[1]):
            t2 = int(t * ratio)
            value = midi_array[t2][i]
            if value > 0:
                resized_array[t][i] = 127
    return resized_array

def show_midi_plot(midi_array):
    midi_array = np.flip(midi_array.T, axis=0)
    f = plot.figure()
    f.set_figwidth(20)
    f.set_figheight(10)
    plot.imshow(midi_array, cmap='binary', interpolation='None', aspect="auto")
    plot.show()

def convert_array_to_midi(np_array, length_seconds, tempo=500_000):
    # get new length of array in ticks
    ticks = int(mido.second2tick(second=length_seconds, ticks_per_beat=480, tempo=tempo))
    # resize array proportionally
    np_array = grow_array_proportionally(np_array, ticks)
    # resize array to midi compatable (88 keys)
    midi_ready_array = np.zeros(shape=(ticks, 88))
    for i in range(NUM_FEATS):
        midi_ready_array[:,i + MIDI_OFFSET] = np_array[:,i]
    # get the difference
    new_ary = np.concatenate([np.array([[0] * 88]), np.array(midi_ready_array, dtype='int64')], axis=0)
    changes = new_ary[1:] - new_ary[:-1]
    # create a midi file with an empty track
    mid_new = mido.MidiFile()
    track = mido.MidiTrack()
    mid_new.tracks.append(track)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))
    # add difference in the empty track
    last_time = 0
    for ch in changes:
        if set(ch) == {0}:  # no change
            last_time += 1
        else:
            on_notes = np.where(ch > 0)[0]
            on_notes_vol = ch[on_notes]
            off_notes = np.where(ch < 0)[0]
            first_ = True
            for n, v in zip(on_notes, on_notes_vol):
                new_time = last_time if first_ else 0
                track.append(mido.Message('note_on', note=n+21, velocity=v, time=new_time))
                first_ = False
            for n in off_notes:
                new_time = last_time if first_ else 0
                track.append(mido.Message('note_off', note=n+21, velocity=0, time=new_time))
                first_ = False
            last_time = 0
    return mid_new

In [ ]:
# 2.0 load arrays from file

data_id = '_15s_100hz'
train_labels = np.load("data/dataset" + data_id + "/train_labels" + data_id + ".npy", allow_pickle=True)

print ("train_labels.shape: ", train_labels.shape)

midi_file = convert_array_to_midi(train_labels[0], length_seconds=15, tempo=500_000)
#midi_file.save('data/midi/test_midi_file.mid')